In [6]:
# import os
# import sys
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import plotly.express as px
import plotly.graph_objects as go


import pandas as pd
import numpy as np
from quantfreedom.utils.caching import clear_cache
from quantfreedom.nb.simulate import backtest_df_array_only
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.evaluators.evaluators import eval_is_below

# np.set_printoptions(formatter={'float_kind':"{:.2f}".format})

# pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv(
    '../data/30min.csv', index_col='time')

rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=15,
)

rsi_eval = eval_is_below(
    rsi_ind,
    np.arange(40, 61, 10),
)

In [7]:
strat_array, settings_array = backtest_df_array_only(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=rsi_eval.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(1,6.1,1),
    risk_rewards=np.arange(1,10.1,1),
    size_pct=1.,
    gains_pct_filter=100,
    # sl_pcts=np.arange(1,6,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(1,6.1,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,5.1,1),
    tsl_when_pct_from_avg_entry=np.arange(1,5.1,1),
)

In [8]:
pd.DataFrame(strat_array).sort_values(by='to_the_upside', ascending=False).head(5)

,or_set,ind_set,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
328,3448,0,43.00,105.54,53.49,0.73,"1,055.41","2,055.41"
327,3447,0,43.00,100.38,51.16,0.70,"1,003.82","2,003.82"
326,3444,1,52.00,100.30,61.54,0.70,"1,002.96","2,002.96"
1819,5193,2,80.00,207.35,47.50,0.70,"2,073.48","3,073.48"
384,3544,2,76.00,149.27,44.74,0.70,"1,492.73","2,492.73"


In [22]:
df_cart = pd.DataFrame(settings_array).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart = df_cart.T
df_cart[326]

order_set_id                        3444
ind_set_id                             1
max_equity_risk_pct                 3.00
risk_rewards                        3.00
size_pct                            1.00
tsl_based_on                   low_price
tsl_pcts_init                       6.00
tsl_trail_by_pct                    4.00
tsl_when_pct_from_avg_entry         5.00
Name: 326, dtype: object

In [23]:
rsi_eval.iloc[:, [1]]

RSI_timeperiod,15
RSI_is_below,50
time,
2020-10-01 00:00:00,False
2020-10-01 00:30:00,False
2020-10-01 01:00:00,False
2020-10-01 01:30:00,False
2020-10-01 02:00:00,False
...,...
2021-02-22 21:30:00,False
2021-02-22 22:00:00,False
